In [13]:
import os
os.chdir("..")

In [14]:
import django
# In case that we need it later
#from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'billboard.settings')
# We need to enable the following because jupyter is running an async loop
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [15]:
import json
import http.client
from rankings.models import Song

In [16]:
from django.conf import settings
api_key = settings.API_KEY

conn = http.client.HTTPSConnection("billboard-api2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': api_key,
    'x-rapidapi-host': "billboard-api2.p.rapidapi.com"
}

conn.request("GET", "/hot-100?date=2019-05-11&range=1-100", headers=headers) # Placeholder date

res = conn.getresponse()
data = res.read()

songs = (data.decode("utf-8"))
songs_data = json.loads(songs)
song_list = songs_data.get('response', [])

In [17]:
from django.utils.dateparse import parse_date

def process_song_data(songs_data):
    for song_key, song_info in songs_data.items():
        title = song_info.get('title')
        artist = song_info.get('artist')
        rank = int(song_info.get('rank'))
        peak_position = int(song_info.get('peak position'))
        weeks_on_chart = int(song_info.get('weeks on chart'))

        chart_date = parse_date('2019-05-11')  # Placeholder date

        song, created = Song.objects.update_or_create(
            title=title,
            artist=artist,
            chart_date=chart_date,
            defaults={
                'rank': rank,
                'album_name': '',
                'peak_position': peak_position,
                'weeks_on_chart': weeks_on_chart,
                'label': '',
                'genre': ''
            }
        )
        if created:
            print(f"Added new song: {title} by {artist}")
        else:
            print(f"Updated song: {title} by {artist}")


In [18]:
song_l = songs_data.get('content', [])
print(song_l)
process_song_data(song_l)

{'1': {'rank': '1', 'title': 'Old Town Road', 'artist': 'Lil Nas X Featuring Billy Ray Cyrus', 'weeks at no.1': '5', 'last week': '1', 'peak position': '1', 'weeks on chart': '9', 'detail': 'same'}, '2': {'rank': '2', 'title': 'ME!', 'artist': 'Taylor Swift Featuring Brendon Urie', 'last week': '100', 'peak position': '2', 'weeks on chart': '2', 'detail': 'up'}, '3': {'rank': '3', 'title': 'Wow.', 'artist': 'Post Malone', 'last week': '2', 'peak position': '2', 'weeks on chart': '19', 'detail': 'down'}, '4': {'rank': '4', 'title': 'Sucker', 'artist': 'Jonas Brothers', 'last week': '5', 'peak position': '1', 'weeks on chart': '9', 'detail': 'up'}, '5': {'rank': '5', 'title': 'Sunflower (Spider-Man: Into The Spider-Verse)', 'artist': 'Post Malone & Swae Lee', 'last week': '3', 'peak position': '1', 'weeks on chart': '28', 'detail': 'down'}, '6': {'rank': '6', 'title': '7 Rings', 'artist': 'Ariana Grande', 'last week': '4', 'peak position': '1', 'weeks on chart': '15', 'detail': 'down'}, 